# 12 layers, pre-trained weight

In [1]:
import tensorflow as tf
import numpy as np
import time
import os
import re

from PIL import Image as PI

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

In [2]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess_ :
    saver = tf.train.import_meta_graph('D:/Backup_data/jongkeun/paper/dogs_cats_190618/dogs_cats_190618-156.meta')
    saver.restore(sess_, tf.train.latest_checkpoint('D:/Backup_data/jongkeun/paper/dogs_cats_190618/'))
    
    graph = tf.get_default_graph()
    
#     print(graph.get_tensor_by_name('layer0_conv2d_1/kernel:0'))
    weight0_1, weight0_2, weight0_3 = sess_.run([graph.get_tensor_by_name('layer0_conv2d_1/kernel:0'), 
                                                 graph.get_tensor_by_name('layer0_conv2d_2/kernel:0'),
                                                 graph.get_tensor_by_name('layer0_conv2d_3/kernel:0')])
    
    weight1, weight1_1, weight1_2, weight1_3 = sess_.run([graph.get_tensor_by_name('layer1_conv2d/kernel:0'), 
                                                          graph.get_tensor_by_name('layer1_conv2d_1/kernel:0'),
                                                          graph.get_tensor_by_name('layer1_conv2d_2/kernel:0'),
                                                          graph.get_tensor_by_name('layer1_conv2d_3/kernel:0')])
    
    weight2, weight2_1, weight2_2, weight2_3 = sess_.run([graph.get_tensor_by_name('layer2_conv2d/kernel:0'), 
                                                          graph.get_tensor_by_name('layer2_conv2d_1/kernel:0'),
                                                          graph.get_tensor_by_name('layer2_conv2d_2/kernel:0'),
                                                          graph.get_tensor_by_name('layer2_conv2d_3/kernel:0')])
    
    weight3, weight3_1, weight3_2, weight3_3 = sess_.run([graph.get_tensor_by_name('layer3_conv2d/kernel:0'), 
                                                          graph.get_tensor_by_name('layer3_conv2d_1/kernel:0'),
                                                          graph.get_tensor_by_name('layer3_conv2d_2/kernel:0'),
                                                          graph.get_tensor_by_name('layer3_conv2d_3/kernel:0')])
#     weight_ = weight[:,:,:,:]
#     print(weight_)

W1107 13:32:46.544084 21324 deprecation.py:323] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [3]:
training_epochs = 500
batch_size = 20
image_resize = 128
image_width = image_resize
image_height = image_resize
initial_rate = 1e-5
label_len = 3

home_dir = "D:/Backup_data/jongkeun"

image_file = "/dir_images/190819_newTraining/"
test_image_file = "/dir_images/190309_new_test/"

label_file = "/csv/190819_newTraining.csv"
test_label_file = "/csv/190309_new_test.csv"

In [4]:
label_dir = home_dir + label_file
test_label_dir = home_dir + test_label_file

# label_dir = os.getcwd() + "/csv/181126.csv"
# test_label_dir = os.getcwd() + "/csv/181126_test.csv"

f_label = open(label_dir, 'r')
f_test_label = open(test_label_dir, 'r')

label_data = f_label.readlines()
test_label_data = f_test_label.readlines()

f_label.close()
f_test_label.close()

label_array = np.array(np.reshape(label_data, [-1, label_len]), dtype=np.int32)
test_label_array = np.array(np.reshape(test_label_data, [-1, label_len]), dtype=np.int32)

print(len(label_array))
print(len(test_label_array))

9182
1018


In [5]:
image_dir = home_dir + image_file
test_image_dir = home_dir + test_image_file

# image_dir = os.getcwd() + "/dir_images/181126/"
# test_image_dir = os.getcwd() + "/dir_images/181126_test/"

image_list = os.listdir(image_dir)
test_image_list = os.listdir(test_image_dir)

image_list = sorted(image_list, key=lambda x: (int(re.sub('\D','',x)),x))
test_image_list = sorted(test_image_list, key=lambda y: (int(re.sub('\D','',y)),y))

image_array = []
test_image_array = []

for i in range(len(image_list)) :
    image_list[i] = image_dir + image_list[i]
    image_array.append(np.array(PI.open(image_list[i]).resize((image_resize, image_resize)).convert('L')))

for j in range(len(test_image_list)) :
    test_image_list[j] = test_image_dir + test_image_list[j]
    test_image_array.append(np.array(PI.open(test_image_list[j]).resize((image_resize, image_resize)).convert('L')))

image_array = np.reshape(image_array, [-1, image_width*image_height])
test_image_array = np.reshape(test_image_array, [-1, image_width*image_height])

image_array = np.array(np.reshape(image_array, [-1, image_width, image_height, 1]), dtype=np.float32)
test_image_array = np.array(np.reshape(test_image_array, [-1, image_width, image_height, 1]), dtype=np.float32)

print(len(image_array))
print(len(test_image_array))

9182
1018


In [6]:
x = tf.placeholder(dtype=tf.float32, shape=[None, image_width, image_height, 1], name='input_image')
y_ = tf.placeholder(dtype=tf.float32, shape=[None, label_len], name='input_label')
learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')

# W_pre_trained = tf.placeholder(dtype=tf.float32, shape=[3,3,8,128], name='pre_trained_weight')

is_training = tf.placeholder(dtype=tf.bool)

In [7]:
W_h0_1 = tf.placeholder(dtype=tf.float32, shape=[3,3,1,8], name='W_h0_1')
W_h0_2 = tf.placeholder(dtype=tf.float32, shape=[3,3,8,128], name='W_h0_2')
W_h0_3 = tf.placeholder(dtype=tf.float32, shape=[3,3,128,256], name='W_h0_3')

W_h1 = tf.placeholder(dtype=tf.float32, shape=[3,3,256,256], name='W_h1')
W_h1_1 = tf.placeholder(dtype=tf.float32, shape=[3,3,256,128], name='W_h1_1')
W_h1_2 = tf.placeholder(dtype=tf.float32, shape=[3,3,128,256], name='W_h1_2')
W_h1_3 = tf.placeholder(dtype=tf.float32, shape=[3,3,256,512], name='W_h1_3')

W_h2 = tf.placeholder(dtype=tf.float32, shape=[3,3,512,512], name='W_h2')
W_h2_1 = tf.placeholder(dtype=tf.float32, shape=[3,3,512,256], name='W_h2_1')
W_h2_2 = tf.placeholder(dtype=tf.float32, shape=[3,3,256,512], name='W_h2_2')
W_h2_3 = tf.placeholder(dtype=tf.float32, shape=[3,3,512,1024], name='W_h2_3')

W_h3 = tf.placeholder(dtype=tf.float32, shape=[3,3,1024,512], name='W_h3')
W_h3_1 = tf.placeholder(dtype=tf.float32, shape=[3,3,512,1024], name='W_h3_1')
W_h3_2 = tf.placeholder(dtype=tf.float32, shape=[3,3,1024,512], name='W_h3_2')
W_h3_3 = tf.placeholder(dtype=tf.float32, shape=[3,3,512,256], name='W_h3_3')

In [8]:
def pooling(input_, dropout, is_training, name_maxpooling, name_dropout) :
#     layer = tf.layers.conv2d(input_data, size, [3, 3], padding='SAME', name=name_conv2d)
    pooling = tf.layers.average_pooling2d(input_, [2, 2], [2, 2], padding='SAME', name=name_maxpooling)
    pooling = tf.layers.dropout(pooling, dropout, is_training, name=name_dropout)
    
    return pooling

def learning_layer(input_data, size, dropout, is_training, name_conv2d, name_maxpooling, name_dropout) :
    layer = tf.layers.conv2d(input_data, size, [3, 3], padding='SAME', name=name_conv2d)
    layer = tf.layers.average_pooling2d(layer, [2, 2], [2, 2], padding='SAME', name=name_maxpooling)
    layer = tf.layers.dropout(layer, dropout, is_training, name=name_dropout)
    
    return layer

def convolutional(input_, weight_) :
    conv_ = tf.nn.conv2d(input_, weight_, strides=[1,1,1,1], padding='SAME')
    hidden_ = tf.nn.relu(conv_)
    
    return hidden_

with tf.device('/gpu:0') :
    layer0_1 = convolutional(x, W_h0_1)
    layer0_2 = convolutional(layer0_1, W_h0_2)
    layer1 = convolutional(layer0_2, W_h0_3)
    layer1_ = pooling(layer1, dropout=0.7, is_training=is_training, name_maxpooling="layer1_maxpooling", name_dropout="layer1_dropout")

    layer1_1 = convolutional(layer1_, W_h1)
    layer1_2 = convolutional(layer1_1, W_h1_1)
    layer2 = convolutional(layer1_2, W_h1_2)
    layer2_ = pooling(layer2, dropout=0.7, is_training=is_training, name_maxpooling="layer2_maxpooling", name_dropout="layer2_dropout")
    
    layer2_1 = convolutional(layer2_, W_h1_3)
    layer2_2 = convolutional(layer2_1, W_h2)
    layer3 = convolutional(layer2_2, W_h2_1)
    layer3_ = pooling(layer3, dropout=0.7, is_training=is_training, name_maxpooling="layer3_maxpooling", name_dropout="layer3_dropout")
    
    layer3_1 = convolutional(layer3_, W_h2_2)
    layer3_2 = convolutional(layer3_1, W_h2_3)
    layer4 = convolutional(layer3_2, W_h3)
    layer4_ = pooling(layer4, dropout=0.7, is_training=is_training, name_maxpooling="layer4_maxpooling", name_dropout="layer4_dropout")

W1107 13:35:12.147768 21324 deprecation.py:323] From <ipython-input-8-ee9d55466c9b>:3: average_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.AveragePooling2D instead.
W1107 13:35:12.277395 21324 deprecation.py:323] From <ipython-input-8-ee9d55466c9b>:4: dropout (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dropout instead.


In [9]:
with tf.device('/gpu:0') :
    fully_layer = tf.contrib.layers.flatten(layer4_)
    fully_layer = tf.layers.dense(fully_layer, 1024, activation=tf.nn.relu, name='fully_connected_layer')
    fully_layer = tf.layers.dropout(fully_layer, 0.5, is_training, name='fully_connected_dropout')

    fully_layer2 = tf.layers.dense(fully_layer, 512, activation=tf.nn.relu, name='fully_connected_layer2')
    fully_layer2 = tf.layers.dropout(fully_layer2, 0.5, is_training, name='fully_connected_dropout2')
    
#     fully_layer3 = tf.layers.dense(fully_layer2, 256, activation=tf.nn.relu, name='fully_connected_layer3')
#     fully_layer3 = tf.layers.dropout(fully_layer3, 0.5, is_training, name='fully_connected_dropout3')

W1107 13:35:15.378102 21324 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1107 13:35:15.380105 21324 deprecation.py:323] From C:\Users\whdrm\Anaconda3\lib\site-packages\tensorflow\contrib\layers\python\layers\layers.py:1634: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W1107 13:35:15.621451 21324 deprecation.py:323] From <ipython-input-9-f6b2f4e2bc49>:3: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W1107 13:35:15.623446 21324 dep

In [10]:
with tf.device('/gpu:0') :
    logits = tf.layers.dense(fully_layer2, label_len, activation=None)
    
    with tf.name_scope('pred') : 
        pred = tf.nn.softmax(logits)
    with tf.name_scope('loss') :
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_))
#         loss = -tf.reduce_sum(y_ * tf.log(pred))
    with tf.name_scope('train') :
        train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

with tf.device('/gpu:0') :
#     correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))
    acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    label_output = tf.argmax(pred, 1)
    label_input = tf.argmax(y_, 1)

In [11]:
config = tf.ConfigProto(log_device_placement = True, allow_soft_placement=True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess :
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess=sess, coord=coord)
        
    sess.run(tf.global_variables_initializer())
    
    total_batch = int(len(image_list) / batch_size)
    correct_list = []
    incorrect_list = []
    cost_list = []
    cost_sum_list = []
    pred_list = []
    total_acc = 0
    cost_sum = 0
    cost_flag = 0
    rate_ = 1
    
    start_time = time.time()
    
    print("")
    
    for epoch in range(training_epochs) :
        total_cost = 0
        
        if epoch%50 == 49 :
                rate_ *= 0.8
                
        if epoch%10 == 9 :
#             rate_ *= 0.8
            midpoint = int(time.time() - start_time)
        
            print("")
            print("=======================================================================================")
            print('{:03d}:{:02d}:{:02d}'.format(midpoint//3600, (midpoint%3600//60), midpoint%60))
            print("=======================================================================================")
            print("")
        
        batch_index = np.random.choice(len(image_array), total_batch, replace=False)
                
        for i in range(total_batch) :                
            _, _loss = sess.run([train, loss], feed_dict={x: image_array[[batch_index[i]]], y_: label_array[[batch_index[i]]], 
                                                          is_training: True, learning_rate: initial_rate*rate_,
                                                          W_h0_1: weight0_1, W_h0_2: weight0_2, W_h0_3: weight0_3,
                                                          W_h1: weight1, W_h1_1: weight1_1, W_h1_2: weight1_2, W_h1_3: weight1_3,
                                                          W_h2: weight2, W_h2_1: weight2_1, W_h2_2: weight2_2, W_h2_3: weight2_3,
                                                          W_h3: weight3, W_h3_1: weight3_1, W_h3_2: weight3_2, W_h3_3: weight3_3})               
            total_cost += _loss

        avg_cost = total_cost/total_batch
        print('Epoch : ', '%4d' % (epoch + 1), '    Avg. cost = ', '{:.4f}'.format(avg_cost))            
        
#         train_writer.add_summary(_merge, epoch)
               
    print("")
    print("=======================================================================================")
    print("================================     Training done     ================================")
    print("=======================================================================================")
    print("")
    
    for test in range(len(test_image_array)) :
        _acc, _label_input, _label_output, _pred = sess.run([acc, label_input, label_output, pred], 
                             feed_dict={x: test_image_array[[test]], y_: test_label_array[[test]], is_training: False,
                                        W_h0_1: weight0_1, W_h0_2: weight0_2, W_h0_3: weight0_3,
                                        W_h1: weight1, W_h1_1: weight1_1, W_h1_2: weight1_2, W_h1_3: weight1_3,
                                        W_h2: weight2, W_h2_1: weight2_1, W_h2_2: weight2_2, W_h2_3: weight2_3,
                                        W_h3: weight3, W_h3_1: weight3_1, W_h3_2: weight3_2, W_h3_3: weight3_3})

        total_acc += _acc
        pred_list.append(list(_pred[0]))
        
        print("n: ", test, "    label_input: ", _label_input, "    label_output: ", _label_output)
        
        if _label_input == _label_output :
            correct_list.append(test)   
        else :
            incorrect_list.append(test)
        
    print("")
    print("          TOTAL ACC    : ", '{:.5f}'.format(total_acc / len(test_image_array)))
    print("")
    print(pred_list)
    
    end_time = int(time.time() - start_time)
    
    print("")
    print("=======================================================================================")
    print('{:03d}:{:02d}:{:02d}'.format(end_time//3600, (end_time%3600//60), end_time%60))
    print("=======================================================================================")
    print("")
    
    coord.request_stop()
    coord.join(thread)

W1107 13:35:16.179958 21324 deprecation.py:323] From <ipython-input-11-500c9753ef23>:6: start_queue_runners (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W1107 13:35:16.180955 21324 queue_runner_impl.py:471] `tf.train.start_queue_runners()` was called when no queue runners were defined. You can safely remove the call to this deprecated function.



Epoch :     1     Avg. cost =  0.8587
Epoch :     2     Avg. cost =  0.7189
Epoch :     3     Avg. cost =  0.6166
Epoch :     4     Avg. cost =  0.6183
Epoch :     5     Avg. cost =  0.5877
Epoch :     6     Avg. cost =  0.5999
Epoch :     7     Avg. cost =  0.5933
Epoch :     8     Avg. cost =  0.6101
Epoch :     9     Avg. cost =  0.6523

000:02:59

Epoch :    10     Avg. cost =  0.5767
Epoch :    11     Avg. cost =  0.6246
Epoch :    12     Avg. cost =  0.5819
Epoch :    13     Avg. cost =  0.6090
Epoch :    14     Avg. cost =  0.5918
Epoch :    15     Avg. cost =  0.5616
Epoch :    16     Avg. cost =  0.6111
Epoch :    17     Avg. cost =  0.5879
Epoch :    18     Avg. cost =  0.5674
Epoch :    19     Avg. cost =  0.5413

000:06:16

Epoch :    20     Avg. cost =  0.5670
Epoch :    21     Avg. cost =  0.6401
Epoch :    22     Avg. cost =  0.5777
Epoch :    23     Avg. cost =  0.5494
Epoch :    24     Avg. cost =  0.5244
Epoch :    25     Avg. cost =  0.5763
Epoch :    26     Avg. co

Epoch :   148     Avg. cost =  0.4269
Epoch :   149     Avg. cost =  0.4384

000:49:04

Epoch :   150     Avg. cost =  0.4433
Epoch :   151     Avg. cost =  0.4088
Epoch :   152     Avg. cost =  0.4765
Epoch :   153     Avg. cost =  0.4697
Epoch :   154     Avg. cost =  0.4129
Epoch :   155     Avg. cost =  0.4701
Epoch :   156     Avg. cost =  0.4370
Epoch :   157     Avg. cost =  0.4766
Epoch :   158     Avg. cost =  0.4689
Epoch :   159     Avg. cost =  0.4145

000:52:22

Epoch :   160     Avg. cost =  0.5005
Epoch :   161     Avg. cost =  0.4238
Epoch :   162     Avg. cost =  0.3926
Epoch :   163     Avg. cost =  0.4182
Epoch :   164     Avg. cost =  0.4216
Epoch :   165     Avg. cost =  0.4169
Epoch :   166     Avg. cost =  0.3957
Epoch :   167     Avg. cost =  0.4328
Epoch :   168     Avg. cost =  0.4241
Epoch :   169     Avg. cost =  0.4385

000:55:39

Epoch :   170     Avg. cost =  0.4373
Epoch :   171     Avg. cost =  0.4839
Epoch :   172     Avg. cost =  0.4403
Epoch :   173 

Epoch :   290     Avg. cost =  0.3881
Epoch :   291     Avg. cost =  0.3755
Epoch :   292     Avg. cost =  0.4139
Epoch :   293     Avg. cost =  0.4544
Epoch :   294     Avg. cost =  0.4348
Epoch :   295     Avg. cost =  0.3764
Epoch :   296     Avg. cost =  0.3635
Epoch :   297     Avg. cost =  0.4302
Epoch :   298     Avg. cost =  0.4310
Epoch :   299     Avg. cost =  0.4517

001:38:32

Epoch :   300     Avg. cost =  0.4369
Epoch :   301     Avg. cost =  0.4415
Epoch :   302     Avg. cost =  0.3477
Epoch :   303     Avg. cost =  0.3843
Epoch :   304     Avg. cost =  0.4202
Epoch :   305     Avg. cost =  0.4156
Epoch :   306     Avg. cost =  0.3958
Epoch :   307     Avg. cost =  0.4398
Epoch :   308     Avg. cost =  0.3829
Epoch :   309     Avg. cost =  0.3520

001:41:50

Epoch :   310     Avg. cost =  0.3791
Epoch :   311     Avg. cost =  0.4424
Epoch :   312     Avg. cost =  0.3927
Epoch :   313     Avg. cost =  0.3925
Epoch :   314     Avg. cost =  0.4036
Epoch :   315     Avg. cos

Epoch :   437     Avg. cost =  0.3702
Epoch :   438     Avg. cost =  0.4156
Epoch :   439     Avg. cost =  0.3557

002:24:46

Epoch :   440     Avg. cost =  0.3594
Epoch :   441     Avg. cost =  0.3426
Epoch :   442     Avg. cost =  0.3746
Epoch :   443     Avg. cost =  0.3981
Epoch :   444     Avg. cost =  0.3609
Epoch :   445     Avg. cost =  0.3763
Epoch :   446     Avg. cost =  0.3599
Epoch :   447     Avg. cost =  0.3735
Epoch :   448     Avg. cost =  0.4334
Epoch :   449     Avg. cost =  0.3531

002:28:04

Epoch :   450     Avg. cost =  0.3845
Epoch :   451     Avg. cost =  0.4070
Epoch :   452     Avg. cost =  0.3673
Epoch :   453     Avg. cost =  0.4235
Epoch :   454     Avg. cost =  0.3696
Epoch :   455     Avg. cost =  0.3708
Epoch :   456     Avg. cost =  0.3925
Epoch :   457     Avg. cost =  0.4181
Epoch :   458     Avg. cost =  0.3855
Epoch :   459     Avg. cost =  0.4124

002:31:22

Epoch :   460     Avg. cost =  0.3390
Epoch :   461     Avg. cost =  0.3613
Epoch :   462 

...

n:  244     label_input:  [0]     label_output:  [1]
n:  245     label_input:  [0]     label_output:  [0]
n:  246     label_input:  [0]     label_output:  [0]
n:  247     label_input:  [0]     label_output:  [0]
n:  248     label_input:  [0]     label_output:  [0]
n:  249     label_input:  [0]     label_output:  [0]
n:  250     label_input:  [0]     label_output:  [0]
n:  251     label_input:  [0]     label_output:  [0]
n:  252     label_input:  [0]     label_output:  [0]
n:  253     label_input:  [0]     label_output:  [1]
n:  254     label_input:  [0]     label_output:  [0]
n:  255     label_input:  [0]     label_output:  [0]
n:  256     label_input:  [0]     label_output:  [0]
n:  257     label_input:  [0]     label_output:  [1]
n:  258     label_input:  [0]     label_output:  [0]
n:  259     label_input:  [0]     label_output:  [0]
n:  260     label_input:  [0]     label_output:  [0]
n:  261     label_input:  [0]     label_output:  [0]
n:  262     label_input:  [0]     label_output

n:  399     label_input:  [1]     label_output:  [1]
n:  400     label_input:  [1]     label_output:  [1]
n:  401     label_input:  [1]     label_output:  [1]
n:  402     label_input:  [1]     label_output:  [1]
n:  403     label_input:  [1]     label_output:  [1]
n:  404     label_input:  [1]     label_output:  [1]
n:  405     label_input:  [1]     label_output:  [1]
n:  406     label_input:  [1]     label_output:  [1]
n:  407     label_input:  [1]     label_output:  [1]
n:  408     label_input:  [1]     label_output:  [1]
n:  409     label_input:  [1]     label_output:  [1]
n:  410     label_input:  [1]     label_output:  [1]
n:  411     label_input:  [1]     label_output:  [0]
n:  412     label_input:  [1]     label_output:  [1]
n:  413     label_input:  [1]     label_output:  [1]
n:  414     label_input:  [1]     label_output:  [1]
n:  415     label_input:  [1]     label_output:  [2]
n:  416     label_input:  [1]     label_output:  [1]
n:  417     label_input:  [1]     label_output

n:  554     label_input:  [1]     label_output:  [1]
n:  555     label_input:  [1]     label_output:  [1]
n:  556     label_input:  [1]     label_output:  [1]
n:  557     label_input:  [1]     label_output:  [1]
n:  558     label_input:  [1]     label_output:  [1]
n:  559     label_input:  [1]     label_output:  [1]
n:  560     label_input:  [1]     label_output:  [1]
n:  561     label_input:  [1]     label_output:  [1]
n:  562     label_input:  [1]     label_output:  [1]
n:  563     label_input:  [1]     label_output:  [1]
n:  564     label_input:  [1]     label_output:  [0]
n:  565     label_input:  [1]     label_output:  [1]
n:  566     label_input:  [1]     label_output:  [1]
n:  567     label_input:  [1]     label_output:  [0]
n:  568     label_input:  [2]     label_output:  [2]
n:  569     label_input:  [2]     label_output:  [2]
n:  570     label_input:  [2]     label_output:  [2]
n:  571     label_input:  [2]     label_output:  [2]
n:  572     label_input:  [2]     label_output

n:  711     label_input:  [2]     label_output:  [0]
n:  712     label_input:  [2]     label_output:  [2]
n:  713     label_input:  [2]     label_output:  [2]
n:  714     label_input:  [2]     label_output:  [2]
n:  715     label_input:  [2]     label_output:  [2]
n:  716     label_input:  [2]     label_output:  [0]
n:  717     label_input:  [2]     label_output:  [2]
n:  718     label_input:  [2]     label_output:  [2]
n:  719     label_input:  [2]     label_output:  [2]
n:  720     label_input:  [2]     label_output:  [2]
n:  721     label_input:  [2]     label_output:  [2]
n:  722     label_input:  [2]     label_output:  [2]
n:  723     label_input:  [2]     label_output:  [2]
n:  724     label_input:  [2]     label_output:  [2]
n:  725     label_input:  [2]     label_output:  [2]
n:  726     label_input:  [2]     label_output:  [1]
n:  727     label_input:  [2]     label_output:  [2]
n:  728     label_input:  [2]     label_output:  [0]
n:  729     label_input:  [2]     label_output

n:  869     label_input:  [2]     label_output:  [2]
n:  870     label_input:  [2]     label_output:  [2]
n:  871     label_input:  [2]     label_output:  [2]
n:  872     label_input:  [2]     label_output:  [2]
n:  873     label_input:  [2]     label_output:  [2]
n:  874     label_input:  [2]     label_output:  [2]
n:  875     label_input:  [2]     label_output:  [1]
n:  876     label_input:  [2]     label_output:  [0]
n:  877     label_input:  [2]     label_output:  [0]
n:  878     label_input:  [2]     label_output:  [0]
n:  879     label_input:  [2]     label_output:  [2]
n:  880     label_input:  [2]     label_output:  [2]
n:  881     label_input:  [2]     label_output:  [2]
n:  882     label_input:  [2]     label_output:  [2]
n:  883     label_input:  [2]     label_output:  [2]
n:  884     label_input:  [2]     label_output:  [2]
n:  885     label_input:  [2]     label_output:  [2]
n:  886     label_input:  [2]     label_output:  [2]
n:  887     label_input:  [2]     label_output